In [ ]:
using System.IO;

string testMessage = "Testing writing some arbitrary string to a stream a second time";
byte[] messageBytes = Encoding.UTF8.GetBytes(testMessage);

using (Stream ioStream = new FileStream("file.txt", FileMode.OpenOrCreate)) {
    if (ioStream.CanWrite) {
        ioStream.Seek(0, SeekOrigin.End);
        ioStream.Write(messageBytes, 0, messageBytes.Length);
    } else {
        Console.WriteLine("Couldn't write to our data stream.");
    }
}
Console.WriteLine("Done!");

In [ ]:
using System.IO;

string testMessage = "medmor ";
byte[] messageBytes = Encoding.UTF8.GetBytes(testMessage);

using (Stream ioStream = new FileStream("file.txt", FileMode.OpenOrCreate)){
    if (ioStream.CanWrite) {
        ioStream.Write(messageBytes, 0, messageBytes.Length);
    }else{
        Console.WriteLine("Couldn't write to our data stream.");
    }
    
    if(ioStream.CanRead){
        byte[] destArray = new byte[10];
        ioStream.Read(destArray, 0, 10);
        string result = Encoding.UTF8.GetString(destArray);
        Console.WriteLine(result);
    }
}

In [ ]:
using Newtonsoft.Json;
using System.IO;
public class ComplexModel {
    public string ComplexModelId { get; set; } = Guid.NewGuid().ToString();
    public int NumberDemonstration { get; set; } = 12354;
    public InnerModel smallInnerModel { get; set; }
    public List<InnerModel> listOfInnerModels { get; set; } = new List<InnerModel>() {
        new InnerModel(),
        new InnerModel()
    };
}
public class InnerModel {
    public string randomId { get; set; } = Guid.NewGuid().ToString();
    public string nonRandomString { get; set; } = "I wrote this here.";
}

ComplexModel testModel = new ComplexModel();
string testMessage = JsonConvert.SerializeObject(testModel);
byte[] messageBytes = Encoding.UTF8.GetBytes(testMessage);
using (Stream ioStream = new FileStream(@"file.json", FileMode.OpenOrCreate)) {
    if (ioStream.CanWrite) {
        ioStream.Write(messageBytes, 0, messageBytes.Length);
    } else {
        Console.WriteLine("Couldn't write to our data stream.");
    }
}
Console.WriteLine("Done!");

In [ ]:
ComplexModel testModel = new ComplexModel();
string testMessage = JsonConvert.SerializeObject(testModel);
using (Stream ioStream = new FileStream(@"file.json", FileMode.OpenOrCreate)) {
    using (StreamWriter sw = new StreamWriter(ioStream)) {
        sw.Write(testMessage);
    }
}
Console.WriteLine("Done!");

In [ ]:
ComplexModel testModel = new ComplexModel();
string testMessage = JsonConvert.SerializeObject(testModel);
using (Stream ioStream = new FileStream("file.json", FileMode.OpenOrCreate)) {
    using (StreamWriter sw = new StreamWriter(ioStream)) {
        sw.Write(testMessage);
        sw.BaseStream.Seek(10, SeekOrigin.Begin);
        sw.Write(testMessage);
    }
}

In [ ]:
async Task LogAsync(string msg, int delay) {
    Console.WriteLine($"Running {msg} task begin");
    await Task.Delay(delay);
    Console.WriteLine(msg);
}

var first = LogAsync("2000", 2000);
await LogAsync("500", 500);
var second = LogAsync("1000", 1000);
await LogAsync("400", 400);
await first;
await second;


In [5]:
#r "nuget: Polly"


Installed Packages Polly, 7.2.3

In [11]:
using System.Net;
using System.Net.Http;
using Polly;

public HttpResponseMessage ExecuteRemoteLookup() {
    if (new Random().Next() % 2 == 0) {
        Console.WriteLine("Retrying connections...");
        throw new WebException("Connection Failure", WebExceptionStatus.ConnectFailure);
    }
    return new HttpResponseMessage();
}

private List<WebExceptionStatus> connectionFailure = new List<WebExceptionStatus>() {
    WebExceptionStatus.ConnectFailure,
    WebExceptionStatus.ConnectionClosed,
    WebExceptionStatus.RequestCanceled,
    WebExceptionStatus.PipelineFailure,
    WebExceptionStatus.SendFailure,
    WebExceptionStatus.KeepAliveFailure,
    WebExceptionStatus.Timeout
};
private List<WebExceptionStatus> resourceAccessFailure = new List<WebExceptionStatus>() {
    WebExceptionStatus.NameResolutionFailure,
    WebExceptionStatus.ProxyNameResolutionFailure,
    WebExceptionStatus.ServerProtocolViolation
};
private List<WebExceptionStatus> securityFailure = new List<WebExceptionStatus>() {
    WebExceptionStatus.SecureChannelFailure,
    WebExceptionStatus.TrustFailure
};

public void ExecuteRemoteLookupWithPolly() {
    Policy connFailurePolicy = Policy
        .Handle<WebException>(x => connectionFailure.Contains(x.Status))
        .RetryForever();
    HttpResponseMessage resp = connFailurePolicy.Execute(() => ExecuteRemoteLookup());
    if (resp.IsSuccessStatusCode) {
        Console.WriteLine("Success!");
    }
}

ExecuteRemoteLookupWithPolly();

Success!
